In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# Get active session from snowflake.snowpark
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
# Exploratory Data Analysis (EDA) for "The Sounds of Home" Dataset

import snowflake.snowpark as snowpark
from snowflake.snowpark import functions as F
from snowflake.snowpark.functions import col, count, sum as sum_, avg, min as min_, max as max_
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set plotting style for dissertation-quality figures
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

def main():
    # Load the dataset
    df = session.table("vitalise_data_light_01")

    # Basic dataset information
    total_rows = df.count()
    print(f"Total number of sound event detections: {total_rows:,}")
    
    # Get unique counts for categorical variables
    unique_houses = df.select("HOUSE_NAME").distinct().count()
    unique_rooms = df.select("ROOM_NAME").distinct().count()  
    unique_classes = df.select("CLASS_NAME").distinct().count()
    unique_parents = df.select("PARENT_NAME").distinct().count()
    unique_files = df.select("FILENAME").distinct().count()
    
    print(f"Number of unique houses: {unique_houses}")
    print(f"Number of unique rooms: {unique_rooms}")
    print(f"Number of unique sound classes: {unique_classes}")
    print(f"Number of unique parent categories: {unique_parents}")
    print(f"Number of unique audio files: {unique_files}")
    
    # Temporal coverage
    date_range = df.select(
        F.min("DATE_COL").alias("START_DATE"),
        F.max("DATE_COL").alias("END_DATE")
    ).collect()[0]
    
    print(f"Recording period: {date_range['START_DATE']} to {date_range['END_DATE']}")
    
    # Basic statistics for numerical columns
    stats = df.select(
        F.min("PROBABILITY").alias("PROB_MIN"),
        F.max("PROBABILITY").alias("PROB_MAX"), 
        F.avg("PROBABILITY").alias("PROB_AVG"),
        F.stddev("PROBABILITY").alias("PROB_STDDEV"),
        F.min("FRAME_INDEX").alias("FRAME_MIN"),
        F.max("FRAME_INDEX").alias("FRAME_MAX"),
        F.avg("FRAME_INDEX").alias("FRAME_AVG")
    ).collect()[0]
    
    print(f"Probability Statistics:")
    print(f"  Range: {stats['PROB_MIN']:.4f} - {stats['PROB_MAX']:.4f}")
    print(f"  Mean: {stats['PROB_AVG']:.4f}")
    print(f"  Standard Deviation: {stats['PROB_STDDEV']:.4f}")
    
    print(f"Frame Index Statistics:")
    print(f"  Range: {stats['FRAME_MIN']} - {stats['FRAME_MAX']}")
    print(f"  Average: {stats['FRAME_AVG']:.2f}")
    
    print("\n" + "="*60)
    print("2. DATA QUALITY ASSESSMENT")
    
    # Check for missing values in all key columns
    null_counts = df.select(
        F.sum(F.when(F.col("CLASS_NAME").isNull(), 1).otherwise(0)).alias("CLASS_NAME_NULLS"),
        F.sum(F.when(F.col("PROBABILITY").isNull(), 1).otherwise(0)).alias("PROBABILITY_NULLS"),
        F.sum(F.when(F.col("HOUSE_NAME").isNull(), 1).otherwise(0)).alias("HOUSE_NAME_NULLS"),
        F.sum(F.when(F.col("ROOM_NAME").isNull(), 1).otherwise(0)).alias("ROOM_NAME_NULLS"),
        F.sum(F.when(F.col("DATE_COL").isNull(), 1).otherwise(0)).alias("DATE_COL_NULLS"),
        F.sum(F.when(F.col("PARENT_NAME").isNull(), 1).otherwise(0)).alias("PARENT_NAME_NULLS")
    ).collect()[0]
    
    print("Missing Values Analysis:")
    for col_name, null_count in null_counts.as_dict().items():
        percentage = (null_count / total_rows) * 100
        print(f"  {col_name}: {null_count} ({percentage:.2f}%)")
    
    # Check probability distribution bounds (should be 0-1)
    prob_bounds_check = df.select(
        F.sum(F.when(F.col("PROBABILITY") < 0, 1).otherwise(0)).alias("NEGATIVE_PROBS"),
        F.sum(F.when(F.col("PROBABILITY") > 1, 1).otherwise(0)).alias("OVER_ONE_PROBS")
    ).collect()[0]
    
    print(f"Probability Bounds Check:")
    print(f"  Values < 0: {prob_bounds_check['NEGATIVE_PROBS']}")
    print(f"  Values > 1: {prob_bounds_check['OVER_ONE_PROBS']}")
    
    print("\n" + "="*60)
    print("3. SPATIAL DISTRIBUTION ANALYSIS")
    
    # House-level analysis
    house_stats = df.group_by("HOUSE_NAME").agg(
        F.count("*").alias("DETECTION_COUNT"),
        F.avg("PROBABILITY").alias("AVG_PROBABILITY"),
        F.count_distinct("CLASS_NAME").alias("UNIQUE_CLASSES"),
        F.count_distinct("ROOM_NAME").alias("ROOMS_COUNT")
    ).order_by("HOUSE_NAME").collect()
    
    print("House-level Statistics:")
    print("House\t\tDetections\tAvg_Prob\tUnique_Classes\tRooms")
    print("*" * 65)
    for row in house_stats:
        print(f"{row['HOUSE_NAME']}\t\t{row['DETECTION_COUNT']:,}\t\t{row['AVG_PROBABILITY']:.3f}\t\t{row['UNIQUE_CLASSES']}\t\t{row['ROOMS_COUNT']}")
    
    # Room-level analysis
    room_stats = df.group_by("ROOM_NAME").agg(
        F.count("*").alias("DETECTION_COUNT"),
        F.avg("PROBABILITY").alias("AVG_PROBABILITY"),
        F.count_distinct("CLASS_NAME").alias("UNIQUE_CLASSES"),
        F.count_distinct("HOUSE_NAME").alias("HOUSE_COUNT")
    ).order_by(F.desc("DETECTION_COUNT")).collect()
    
    print(f"\nRoom-level Statistics (Top 10):")
    print("Room\t\t\tDetections\tAvg_Prob\tUnique_Classes\tHouses")
    print("*" * 70)
    for row in room_stats[:10]:
        print(f"{row['ROOM_NAME']:<15}\t{row['DETECTION_COUNT']:,}\t\t{row['AVG_PROBABILITY']:.3f}\t\t{row['UNIQUE_CLASSES']}\t\t{row['HOUSE_COUNT']}")
    
    print("\n" + "*"*60)
    print("4. SOUND EVENT CLASS ANALYSIS")
    print("*"*60)
    
    # Most frequent sound classes
    class_freq = df.group_by("CLASS_NAME").agg(
        F.count("*").alias("FREQUENCY"),
        F.avg("PROBABILITY").alias("AVG_CONFIDENCE"),
        F.count_distinct("HOUSE_NAME").alias("HOUSE_COVERAGE"),
        F.count_distinct("ROOM_NAME").alias("ROOM_COVERAGE")
    ).order_by(F.desc("FREQUENCY")).collect()
    
    print("Top 15 Most Frequent Sound Classes:")
    print("Sound Class\t\t\tFrequency\tAvg_Confidence\tHouse_Coverage\tRoom_Coverage")
    print("*" * 85)
    for i, row in enumerate(class_freq[:15]):
        freq_pct = (row['FREQUENCY'] / total_rows) * 100
        print(f"{row['CLASS_NAME']:<20}\t{row['FREQUENCY']:,} ({freq_pct:.1f}%)\t{row['AVG_CONFIDENCE']:.3f}\t\t{row['HOUSE_COVERAGE']}\t\t{row['ROOM_COVERAGE']}")
    
    # Parent category analysis (filter out null values)
    parent_stats = df.filter(F.col("PARENT_NAME").isNotNull()).group_by("PARENT_NAME").agg(
        F.count("*").alias("FREQUENCY"),
        F.avg("PROBABILITY").alias("AVG_CONFIDENCE"),
        F.count_distinct("CLASS_NAME").alias("CHILD_CLASSES")
    ).order_by(F.desc("FREQUENCY")).collect()
    
    # Also check how many records have null parent names
    null_parent_count = df.filter(F.col("PARENT_NAME").isNull()).count()
    null_parent_pct = (null_parent_count / total_rows) * 100
    
    print(f"Parent Category Analysis (Top 10):")
    if null_parent_count > 0:
        print(f"Note: {null_parent_count:,} records ({null_parent_pct:.1f}%) have null parent categories")
    print("Parent Category\t\t\tFrequency\tAvg_Confidence\tChild_Classes")
    print("*" * 70)
    for row in parent_stats[:10]:
        freq_pct = (row['FREQUENCY'] / total_rows) * 100
        parent_name = row['PARENT_NAME'] if row['PARENT_NAME'] is not None else "NULL/UNKNOWN"
        print(f"{parent_name:<25}\t{row['FREQUENCY']:,} ({freq_pct:.1f}%)\t{row['AVG_CONFIDENCE']:.3f}\t\t{row['CHILD_CLASSES']}")
    
    print("\n" + "*"*60)
    print("5. TEMPORAL PATTERN ANALYSIS")
    print("*"*60)
    
    # Daily pattern analysis
    daily_pattern = df.group_by("DATE_COL").agg(
        F.count("*").alias("TOTAL_DETECTIONS")
    ).order_by("DATE_COL").collect()
    
    print("Daily Detection Patterns:")
    print("Date\t\t\tDetections")
    print("*" * 30)
    for row in daily_pattern:
        print(f"{row['DATE_COL']}\t\t{row['TOTAL_DETECTIONS']:,}")
    
    # Hourly pattern analysis (extract hour from TIME_COL)
    hourly_pattern = df.select(
        F.hour(F.to_time("TIME_COL")).alias("HOUR"),
        "*"
    ).group_by("HOUR").agg(
        F.count("*").alias("HOURLY_DETECTIONS"),
        F.avg("PROBABILITY").alias("AVG_HOURLY_CONFIDENCE")
    ).order_by("HOUR").collect()
    
    print(f"\nHourly Activity Pattern:")
    print("Hour\tDetections\tAvg_Confidence")
    print("-" * 35)
    for row in hourly_pattern:
        if row['HOUR'] is not None:
            print(f"{row['HOUR']:02d}:00\t{row['HOURLY_DETECTIONS']:,}\t\t{row['AVG_HOURLY_CONFIDENCE']:.3f}")
    
    print("\n" + "="*60)
    print("6. CONFIDENCE SCORE ANALYSIS")
    print("="*60)
    
    # Probability distribution analysis
    prob_quantiles = df.select(
        F.expr("PERCENTILE_CONT(0.1) WITHIN GROUP (ORDER BY PROBABILITY)").alias("P10"),
        F.expr("PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY PROBABILITY)").alias("P25"), 
        F.expr("PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY PROBABILITY)").alias("P50"),
        F.expr("PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY PROBABILITY)").alias("P75"),
        F.expr("PERCENTILE_CONT(0.9) WITHIN GROUP (ORDER BY PROBABILITY)").alias("P90"),
        F.expr("PERCENTILE_CONT(0.95) WITHIN GROUP (ORDER BY PROBABILITY)").alias("P95"),
        F.expr("PERCENTILE_CONT(0.99) WITHIN GROUP (ORDER BY PROBABILITY)").alias("P99")
    ).collect()[0]
    
    print("Confidence Score Distribution (Quantiles):")
    for percentile, value in prob_quantiles.as_dict().items():
        print(f"  {percentile}: {value:.4f}")
    
    # High confidence detections (>= 0.8)
    high_conf_analysis = df.filter(F.col("PROBABILITY") >= 0.8).group_by("CLASS_NAME").agg(
        F.count("*").alias("HIGH_CONF_COUNT")
    ).order_by(F.desc("HIGH_CONF_COUNT")).collect()
    
    high_conf_total = sum([row['HIGH_CONF_COUNT'] for row in high_conf_analysis])
    high_conf_percentage = (high_conf_total / total_rows) * 100
    
    print(f"\nHigh Confidence Detections (>= 0.8): {high_conf_total:,} ({high_conf_percentage:.1f}%)")
    print("Top 10 Sound Classes with High Confidence:")
    print("Sound Class\t\t\tHigh Conf Count\t% of Total High Conf")
    print("-" * 60)
    for row in high_conf_analysis[:10]:
        pct_of_high_conf = (row['HIGH_CONF_COUNT'] / high_conf_total) * 100
        print(f"{row['CLASS_NAME']:<25}\t{row['HIGH_CONF_COUNT']:,}\t\t{pct_of_high_conf:.1f}%")
    
    print("\n" + "*"*60)
    print("7. RESEARCH INSIGHTS AND IMPLICATIONS")
    print("*"*60)
    
    # Calculate key metrics for research discussion
    avg_detections_per_file = total_rows / unique_files
    avg_classes_per_house = sum([row['UNIQUE_CLASSES'] for row in house_stats]) / len(house_stats)
    
    print("Key Research Metrics:")
    print(f"  Average detections per audio file: {avg_detections_per_file:.1f}")
    print(f"  Average unique sound classes per house: {avg_classes_per_house:.1f}")
    print(f"  Dataset density: {total_rows / (unique_houses * unique_files):.1f} detections per house-file combination")
    
    # Identify most distinctive sounds per room type
    print(f"\nRoom-Specific Sound Characteristics:")
    room_specific_sounds = df.group_by("ROOM_NAME", "CLASS_NAME").agg(
        F.count("*").alias("COUNT")
    ).group_by("ROOM_NAME").agg(
        F.max("COUNT").alias("MAX_COUNT"),
        F.sum("COUNT").alias("TOTAL_COUNT")
    ).collect()
    
    for room_stat in room_specific_sounds[:5]:
        dominance_ratio = room_stat['MAX_COUNT'] / room_stat['TOTAL_COUNT']
        print(f"  {room_stat['ROOM_NAME']}: Sound dominance ratio = {dominance_ratio:.3f}")
    
    print(f"\nDataset Quality Indicators:")
    print(f"  Completeness: {100 - (sum(null_counts.as_dict().values()) / (total_rows * len(null_counts.as_dict()))) * 100:.1f}%")
    print(f"  Confidence reliability: {high_conf_percentage:.1f}% high-confidence detections")
    print(f"  Spatial coverage: {unique_houses} homes, {unique_rooms} room types")
    print(f"  Temporal coverage: Multi-day recordings with {len(daily_pattern)} recording days")
    
    
    print("\n" + "*"*80)
    print("EDA COMPLETE - Dataset is ready for advanced machine learning analysis")
    print("*"*80)

# Additional utility functions for extended analysis
def export_summary_statistics():
    """Export key statistics for dissertation appendix"""
    df = session.table("vitalise_data_light_01")
    
    # Create comprehensive summary table
    summary_stats = df.select(
        F.count("*").alias("total_records"),
        F.count_distinct("HOUSE_NAME").alias("unique_houses"),
        F.count_distinct("ROOM_NAME").alias("unique_rooms"),
        F.count_distinct("CLASS_NAME").alias("unique_sound_classes"),
        F.min("PROBABILITY").alias("min_confidence"),
        F.max("PROBABILITY").alias("max_confidence"),
        F.avg("PROBABILITY").alias("mean_confidence"),
        F.stddev("PROBABILITY").alias("std_confidence")
    ).collect()
    
    return summary_stats

def analyze_sound_event_sequences():
    """Analyse sequential patterns in sound events - useful for ADL research"""
    df = session.table("vitalise_data_light_01")
    
    # Get sequences of sound events within each file
    sequences = df.select(
        "FILENAME", "HOUSE_NAME", "ROOM_NAME", 
        "FRAME_INDEX", "CLASS_NAME", "PROBABILITY"
    ).order_by("FILENAME", "FRAME_INDEX").collect()
    
    return sequences

# Run the main EDA function
if __name__ == "__main__":
    main()

In [ ]:
# Comprehensive Exploratory Data Analysis (EDA) for "The Sounds of Home" Dataset

import snowflake.snowpark as snowpark
from snowflake.snowpark import functions as F
from snowflake.snowpark.functions import col, count, sum as sum_, avg, min as min_, max as max_
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set plotting style for dissertation-quality figures
plt.style.use('default')
sns.set_style("whitegrid")
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10
plt.rcParams['legend.fontsize'] = 10

def main():
    """
    Main EDA function for The Sounds of Home dataset
    
    This analysis focuses on:
    1. Dataset overview and completeness
    2. Temporal patterns in domestic soundscapes
    3. Sound event classification analysis
    4. Spatial distribution across homes and rooms
    5. Probability distribution analysis
    6. Data quality assessment
    """
    
    # Load the dataset
    df = session.table("vitalise_data_light_01")
    
    # Basic dataset information
    total_rows = df.count()
    print(f"Total number of sound event detections: {total_rows:,}")
    
    # Get unique counts for categorical variables
    unique_houses = df.select("HOUSE_NAME").distinct().count()
    unique_rooms = df.select("ROOM_NAME").distinct().count()  
    unique_classes = df.select("CLASS_NAME").distinct().count()
    unique_parents = df.select("PARENT_NAME").distinct().count()
    unique_files = df.select("FILENAME").distinct().count()
    
    print(f"Number of unique houses: {unique_houses}")
    print(f"Number of unique rooms: {unique_rooms}")
    print(f"Number of unique sound classes: {unique_classes}")
    print(f"Number of unique parent categories: {unique_parents}")
    print(f"Number of unique audio files: {unique_files}")
    
    # Temporal coverage
    date_range = df.select(
        F.min("DATE_COL").alias("START_DATE"),
        F.max("DATE_COL").alias("END_DATE")
    ).collect()[0]
    
    print(f"Recording period: {date_range['START_DATE']} to {date_range['END_DATE']}")
    
    # Basic statistics for numerical columns
    stats = df.select(
        F.min("PROBABILITY").alias("PROB_MIN"),
        F.max("PROBABILITY").alias("PROB_MAX"), 
        F.avg("PROBABILITY").alias("PROB_AVG"),
        F.stddev("PROBABILITY").alias("PROB_STDDEV"),
        F.min("FRAME_INDEX").alias("FRAME_MIN"),
        F.max("FRAME_INDEX").alias("FRAME_MAX"),
        F.avg("FRAME_INDEX").alias("FRAME_AVG")
    ).collect()[0]
    
    print(f"Probability Statistics:")
    print(f"  Range: {stats['PROB_MIN']:.4f} - {stats['PROB_MAX']:.4f}")
    print(f"  Mean: {stats['PROB_AVG']:.4f}")
    print(f"  Standard Deviation: {stats['PROB_STDDEV']:.4f}")
    
    print(f"Frame Index Statistics:")
    print(f"  Range: {stats['FRAME_MIN']} - {stats['FRAME_MAX']}")
    print(f"  Average: {stats['FRAME_AVG']:.2f}")
    
    print("2. DATA QUALITY ASSESSMENT")
    
    # Check for missing values in all key columns
    null_counts = df.select(
        F.sum(F.when(F.col("CLASS_NAME").isNull(), 1).otherwise(0)).alias("CLASS_NAME_NULLS"),
        F.sum(F.when(F.col("PROBABILITY").isNull(), 1).otherwise(0)).alias("PROBABILITY_NULLS"),
        F.sum(F.when(F.col("HOUSE_NAME").isNull(), 1).otherwise(0)).alias("HOUSE_NAME_NULLS"),
        F.sum(F.when(F.col("ROOM_NAME").isNull(), 1).otherwise(0)).alias("ROOM_NAME_NULLS"),
        F.sum(F.when(F.col("DATE_COL").isNull(), 1).otherwise(0)).alias("DATE_COL_NULLS"),
        F.sum(F.when(F.col("PARENT_NAME").isNull(), 1).otherwise(0)).alias("PARENT_NAME_NULLS")
    ).collect()[0]
    
    print("Missing Values Analysis:")
    for col_name, null_count in null_counts.as_dict().items():
        percentage = (null_count / total_rows) * 100
        print(f"  {col_name}: {null_count} ({percentage:.2f}%)")
    
    # Check probability distribution bounds (should be 0-1)
    prob_bounds_check = df.select(
        F.sum(F.when(F.col("PROBABILITY") < 0, 1).otherwise(0)).alias("NEGATIVE_PROBS"),
        F.sum(F.when(F.col("PROBABILITY") > 1, 1).otherwise(0)).alias("OVER_ONE_PROBS")
    ).collect()[0]
    
    print(f"Probability Bounds Check:")
    print(f"  Values < 0: {prob_bounds_check['NEGATIVE_PROBS']}")
    print(f"  Values > 1: {prob_bounds_check['OVER_ONE_PROBS']}")
    

    print("3. SPATIAL DISTRIBUTION ANALYSIS")

    
    # House-level analysis
    house_stats = df.group_by("HOUSE_NAME").agg(
        F.count("*").alias("DETECTION_COUNT"),
        F.avg("PROBABILITY").alias("AVG_PROBABILITY"),
        F.count_distinct("CLASS_NAME").alias("UNIQUE_CLASSES"),
        F.count_distinct("ROOM_NAME").alias("ROOMS_COUNT")
    ).order_by("HOUSE_NAME").collect()
    
    print("House-level Statistics:")
    print("House\t\tDetections\tAvg_Prob\tUnique_Classes\tRooms")
    print("*" * 65)
    for row in house_stats:
        print(f"{row['HOUSE_NAME']}\t\t{row['DETECTION_COUNT']:,}\t\t{row['AVG_PROBABILITY']:.3f}\t\t{row['UNIQUE_CLASSES']}\t\t{row['ROOMS_COUNT']}")
    
    # Room-level analysis
    room_stats = df.group_by("ROOM_NAME").agg(
        F.count("*").alias("DETECTION_COUNT"),
        F.avg("PROBABILITY").alias("AVG_PROBABILITY"),
        F.count_distinct("CLASS_NAME").alias("UNIQUE_CLASSES"),
        F.count_distinct("HOUSE_NAME").alias("HOUSE_COUNT")
    ).order_by(F.desc("DETECTION_COUNT")).collect()
    
    print(f"Room-level Statistics (Top 10):")
    print("Room\t\t\tDetections\tAvg_Prob\tUnique_Classes\tHouses")
    print("*" * 70)
    for row in room_stats[:10]:
        print(f"{row['ROOM_NAME']:<15}\t{row['DETECTION_COUNT']:,}\t\t{row['AVG_PROBABILITY']:.3f}\t\t{row['UNIQUE_CLASSES']}\t\t{row['HOUSE_COUNT']}")
    
    print("4. SOUND EVENT CLASS ANALYSIS")

    
    # Most frequent sound classes
    class_freq = df.group_by("CLASS_NAME").agg(
        F.count("*").alias("FREQUENCY"),
        F.avg("PROBABILITY").alias("AVG_CONFIDENCE"),
        F.count_distinct("HOUSE_NAME").alias("HOUSE_COVERAGE"),
        F.count_distinct("ROOM_NAME").alias("ROOM_COVERAGE")
    ).order_by(F.desc("FREQUENCY")).collect()
    
    print("Top 15 Most Frequent Sound Classes:")
    print("Sound Class\t\t\tFrequency\tAvg_Confidence\tHouse_Coverage\tRoom_Coverage")
    print("*" * 85)
    for i, row in enumerate(class_freq[:15]):
        freq_pct = (row['FREQUENCY'] / total_rows) * 100
        print(f"{row['CLASS_NAME']:<20}\t{row['FREQUENCY']:,} ({freq_pct:.1f}%)\t{row['AVG_CONFIDENCE']:.3f}\t\t{row['HOUSE_COVERAGE']}\t\t{row['ROOM_COVERAGE']}")
    
    # Parent category analysis (filter out null values)
    parent_stats = df.filter(F.col("PARENT_NAME").isNotNull()).group_by("PARENT_NAME").agg(
        F.count("*").alias("FREQUENCY"),
        F.avg("PROBABILITY").alias("AVG_CONFIDENCE"),
        F.count_distinct("CLASS_NAME").alias("CHILD_CLASSES")
    ).order_by(F.desc("FREQUENCY")).collect()
    
    # checking how many records have null parent names
    null_parent_count = df.filter(F.col("PARENT_NAME").isNull()).count()
    null_parent_pct = (null_parent_count / total_rows) * 100
    
    print(f"\nParent Category Analysis (Top 10):")
    if null_parent_count > 0:
        print(f"Note: {null_parent_count:,} records ({null_parent_pct:.1f}%) have null parent categories")
    print("Parent Category\t\t\tFrequency\tAvg_Confidence\tChild_Classes")
    print("*" * 70)
    for row in parent_stats[:10]:
        freq_pct = (row['FREQUENCY'] / total_rows) * 100
        parent_name = row['PARENT_NAME'] if row['PARENT_NAME'] is not None else "NULL/UNKNOWN"
        print(f"{parent_name:<25}\t{row['FREQUENCY']:,} ({freq_pct:.1f}%)\t{row['AVG_CONFIDENCE']:.3f}\t\t{row['CHILD_CLASSES']}")
    
    print("5. TEMPORAL PATTERN ANALYSIS")

    
    # Daily pattern analysis
    daily_pattern = df.group_by("DATE_COL").agg(
        F.count("*").alias("TOTAL_DETECTIONS")
    ).order_by("DATE_COL").collect()
    
    print("Daily Detection Patterns:")
    print("Date\t\t\tDetections")
    print("*" * 30)
    for row in daily_pattern:
        print(f"{row['DATE_COL']}\t\t{row['TOTAL_DETECTIONS']:,}")
    
    # Hourly pattern analysis (extract hour from TIME_COL)
    hourly_pattern = df.select(
        F.hour(F.to_time("TIME_COL")).alias("HOUR"),
        "*"
    ).group_by("HOUR").agg(
        F.count("*").alias("HOURLY_DETECTIONS"),
        F.avg("PROBABILITY").alias("AVG_HOURLY_CONFIDENCE")
    ).order_by("HOUR").collect()
    
    print(f"\nHourly Activity Pattern:")
    print("Hour\tDetections\tAvg_Confidence")
    print("*" * 35)
    for row in hourly_pattern:
        if row['HOUR'] is not None:
            print(f"{row['HOUR']:02d}:00\t{row['HOURLY_DETECTIONS']:,}\t\t{row['AVG_HOURLY_CONFIDENCE']:.3f}")
    
    print("6. CONFIDENCE SCORE ANALYSIS")

    # Probability distribution analysis
    prob_quantiles = df.select(
        F.expr("PERCENTILE_CONT(0.1) WITHIN GROUP (ORDER BY PROBABILITY)").alias("P10"),
        F.expr("PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY PROBABILITY)").alias("P25"), 
        F.expr("PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY PROBABILITY)").alias("P50"),
        F.expr("PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY PROBABILITY)").alias("P75"),
        F.expr("PERCENTILE_CONT(0.9) WITHIN GROUP (ORDER BY PROBABILITY)").alias("P90"),
        F.expr("PERCENTILE_CONT(0.95) WITHIN GROUP (ORDER BY PROBABILITY)").alias("P95"),
        F.expr("PERCENTILE_CONT(0.99) WITHIN GROUP (ORDER BY PROBABILITY)").alias("P99")
    ).collect()[0]
    
    print("Confidence Score Distribution (Quantiles):")
    for percentile, value in prob_quantiles.as_dict().items():
        print(f"  {percentile}: {value:.4f}")
    
    # High confidence detections (>= 0.8)
    high_conf_analysis = df.filter(F.col("PROBABILITY") >= 0.8).group_by("CLASS_NAME").agg(
        F.count("*").alias("HIGH_CONF_COUNT")
    ).order_by(F.desc("HIGH_CONF_COUNT")).collect()
    
    high_conf_total = sum([row['HIGH_CONF_COUNT'] for row in high_conf_analysis])
    high_conf_percentage = (high_conf_total / total_rows) * 100
    
    print(f"\nHigh Confidence Detections (>= 0.8): {high_conf_total:,} ({high_conf_percentage:.1f}%)")
    print("Top 10 Sound Classes with High Confidence:")
    print("Sound Class\t\t\tHigh Conf Count\t% of Total High Conf")
    print("*" * 60)
    for row in high_conf_analysis[:10]:
        pct_of_high_conf = (row['HIGH_CONF_COUNT'] / high_conf_total) * 100
        print(f"{row['CLASS_NAME']:<25}\t{row['HIGH_CONF_COUNT']:,}\t\t{pct_of_high_conf:.1f}%")
    

    print("7. RESEARCH INSIGHTS AND IMPLICATIONS")

    # Calculate key metrics for research discussion
    avg_detections_per_file = total_rows / unique_files
    avg_classes_per_house = sum([row['UNIQUE_CLASSES'] for row in house_stats]) / len(house_stats)
    
    print("Key Research Metrics:")
    print(f"  Average detections per audio file: {avg_detections_per_file:.1f}")
    print(f"  Average unique sound classes per house: {avg_classes_per_house:.1f}")
    print(f"  Dataset density: {total_rows / (unique_houses * unique_files):.1f} detections per house-file combination")
    
    # Identify most distinctive sounds per room type
    print(f"\nRoom-Specific Sound Characteristics:")
    room_specific_sounds = df.group_by("ROOM_NAME", "CLASS_NAME").agg(
        F.count("*").alias("COUNT")
    ).group_by("ROOM_NAME").agg(
        F.max("COUNT").alias("MAX_COUNT"),
        F.sum("COUNT").alias("TOTAL_COUNT")
    ).collect()
    
    for room_stat in room_specific_sounds[:5]:
        dominance_ratio = room_stat['MAX_COUNT'] / room_stat['TOTAL_COUNT']
        print(f"  {room_stat['ROOM_NAME']}: Sound dominance ratio = {dominance_ratio:.3f}")
    
    print(f"Dataset Quality Indicators:")
    print(f"  Completeness: {100 - (sum(null_counts.as_dict().values()) / (total_rows * len(null_counts.as_dict()))) * 100:.1f}%")
    print(f"  Confidence reliability: {high_conf_percentage:.1f}% high-confidence detections")
    print(f"  Spatial coverage: {unique_houses} homes, {unique_rooms} room types")
    print(f"  Temporal coverage: Multi-day recordings with {len(daily_pattern)} recording days")

    print("EDA COMPLETE - Dataset is ready for advanced machine learning analysis")
    
    # Generate comprehensive visualizations
    create_visualizations(df, total_rows)

def create_visualizations(df, total_rows):
    """
    Create comprehensive visualisations
    """
    
    plt.ioff()  # Turn off interactive mode for batch processing
    
    # 1. TEMPORAL PATTERNS VISUALISATION
    print("1. Creating temporal pattern visualisations...")
    
    # Daily activity heatmap by hour
    hourly_data = df.select(
        F.hour(F.to_time("TIME_COL")).alias("HOUR"),
        "DATE_COL"
    ).filter(F.col("HOUR").isNotNull()).group_by("DATE_COL", "HOUR").agg(
        F.count("*").alias("DETECTIONS")
    ).collect()
    
    if hourly_data:
        # Convert to pandas for visualization
        hourly_df = pd.DataFrame([row.asDict() for row in hourly_data])
        hourly_pivot = hourly_df.pivot(index='DATE_COL', columns='HOUR', values='DETECTIONS')
        hourly_pivot = hourly_pivot.fillna(0)
        
        # Create heatmap
        fig, ax = plt.subplots(figsize=(15, 6))
        sns.heatmap(hourly_pivot, annot=False, cmap='YlOrRd', cbar_kws={'label': 'Number of Detections'})
        plt.title('Daily Activity Patterns: Sound Events by Hour and Date\nThe Sounds of Home Dataset', 
                 fontsize=16, fontweight='bold', pad=20)
        plt.xlabel('Hour of Day', fontsize=12)
        plt.ylabel('Recording Date', fontsize=12)
        plt.tight_layout()
        plt.savefig('temporal_heatmap.png', dpi=300, bbox_inches='tight')
        plt.show()
    
    # 2. SOUND CLASS DISTRIBUTION
    print("2. Creating sound class distribution visualizations...")
    
    # Top sound classes
    top_classes = df.group_by("CLASS_NAME").agg(
        F.count("*").alias("FREQUENCY"),
        F.avg("PROBABILITY").alias("AVG_CONFIDENCE")
    ).order_by(F.desc("FREQUENCY")).limit(20).collect()
    
    if top_classes:
        classes_df = pd.DataFrame([row.asDict() for row in top_classes])
        classes_df['PERCENTAGE'] = (classes_df['FREQUENCY'] / total_rows) * 100
        
        # Create bar plot
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))
        
        # Frequency plot
        sns.barplot(data=classes_df.head(15), y='CLASS_NAME', x='FREQUENCY', 
                   palette='viridis', ax=ax1)
        ax1.set_title('Top 15 Most Frequent Sound Classes', fontsize=14, fontweight='bold')
        ax1.set_xlabel('Number of Detections (log scale)', fontsize=12)
        ax1.set_ylabel('Sound Class', fontsize=12)
        ax1.set_xscale('log')
        ax1.tick_params(axis='y', labelsize=10)
        
        # Confidence vs Frequency scatter
        sns.scatterplot(data=classes_df, x='FREQUENCY', y='AVG_CONFIDENCE', 
                       size='PERCENTAGE', alpha=0.7, ax=ax2)
        ax2.set_title('Sound Class Confidence vs Frequency', fontsize=14, fontweight='bold')
        ax2.set_xlabel('Detection Frequency (log scale)', fontsize=12)
        ax2.set_ylabel('Average Confidence Score', fontsize=12)
        ax2.set_xscale('log')
        ax2.legend(title='% of Dataset', bbox_to_anchor=(1.05, 1), loc='upper left')
        
        plt.tight_layout()
        plt.savefig('sound_class_analysis.png', dpi=300, bbox_inches='tight')
        plt.show()
    
    # 3. CONFIDENCE SCORE DISTRIBUTIONS
    print("3. Creating confidence score distribution visualizations...")
    
    # Sample confidence scores for visualization (sample for performance)
    confidence_sample = df.select("PROBABILITY", "CLASS_NAME").sample(0.01).collect()
    
    if confidence_sample:
        conf_df = pd.DataFrame([row.asDict() for row in confidence_sample])
        
        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))
        
        # Overall distribution
        ax1.hist(conf_df['PROBABILITY'], bins=50, alpha=0.7, color='skyblue', edgecolor='black')
        ax1.axvline(conf_df['PROBABILITY'].mean(), color='red', linestyle='--', 
                   label=f'Mean: {conf_df["PROBABILITY"].mean():.3f}')
        ax1.set_title('Overall Confidence Score Distribution', fontsize=12, fontweight='bold')
        ax1.set_xlabel('Confidence Score', fontsize=10)
        ax1.set_ylabel('Frequency', fontsize=10)
        ax1.legend()
        
        # Box plot by confidence ranges
        conf_df['CONFIDENCE_RANGE'] = pd.cut(conf_df['PROBABILITY'], 
                                           bins=[0, 0.2, 0.4, 0.6, 0.8, 1.0],
                                           labels=['Very Low', 'Low', 'Medium', 'High', 'Very High'])
        conf_range_counts = conf_df['CONFIDENCE_RANGE'].value_counts()
        ax2.pie(conf_range_counts.values, labels=conf_range_counts.index, autopct='%1.1f%%',
               colors=sns.color_palette("husl", len(conf_range_counts)))
        ax2.set_title('Distribution by Confidence Ranges', fontsize=12, fontweight='bold')
        
        # Confidence by top sound classes
        top_classes_conf = conf_df[conf_df['CLASS_NAME'].isin(
            conf_df['CLASS_NAME'].value_counts().head(10).index)]
        sns.boxplot(data=top_classes_conf, x='PROBABILITY', y='CLASS_NAME', ax=ax3)
        ax3.set_title('Confidence Distribution by Top Sound Classes', fontsize=12, fontweight='bold')
        ax3.set_xlabel('Confidence Score', fontsize=10)
        ax3.set_ylabel('Sound Class', fontsize=10)
        
        # Cumulative distribution
        sorted_conf = np.sort(conf_df['PROBABILITY'])
        cumulative = np.arange(1, len(sorted_conf) + 1) / len(sorted_conf)
        ax4.plot(sorted_conf, cumulative, linewidth=2)
        ax4.set_title('Cumulative Confidence Score Distribution', fontsize=12, fontweight='bold')
        ax4.set_xlabel('Confidence Score', fontsize=10)
        ax4.set_ylabel('Cumulative Probability', fontsize=10)
        ax4.grid(True, alpha=0.3)
        
        plt.suptitle('Confidence Score Analysis - The Sounds of Home Dataset', 
                    fontsize=16, fontweight='bold', y=0.98)
        plt.tight_layout()
        plt.savefig('confidence_analysis.png', dpi=300, bbox_inches='tight')
        plt.show()
    
    # 4. SPATIAL DISTRIBUTION
    print("4. Creating spatial distribution visualizations...")
    
    # House comparison
    house_data = df.group_by("HOUSE_NAME").agg(
        F.count("*").alias("TOTAL_DETECTIONS"),
        F.avg("PROBABILITY").alias("AVG_CONFIDENCE"),
        F.count_distinct("CLASS_NAME").alias("UNIQUE_CLASSES")
    ).collect()
    
    if house_data:
        house_df = pd.DataFrame([row.asDict() for row in house_data])
        
        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))
        
        # Total detections by house
        sns.barplot(data=house_df, x='HOUSE_NAME', y='TOTAL_DETECTIONS', 
                   palette='Set2', ax=ax1)
        ax1.set_title('Total Sound Detections by House', fontsize=12, fontweight='bold')
        ax1.set_xlabel('House ID', fontsize=10)
        ax1.set_ylabel('Number of Detections', fontsize=10)
        ax1.tick_params(axis='x', rotation=45)
        
        # Average confidence by house
        sns.barplot(data=house_df, x='HOUSE_NAME', y='AVG_CONFIDENCE', 
                   palette='Set3', ax=ax2)
        ax2.set_title('Average Confidence Score by House', fontsize=12, fontweight='bold')
        ax2.set_xlabel('House ID', fontsize=10)
        ax2.set_ylabel('Average Confidence', fontsize=10)
        ax2.tick_params(axis='x', rotation=45)
        
        # Unique classes by house
        sns.barplot(data=house_df, x='HOUSE_NAME', y='UNIQUE_CLASSES', 
                   palette='Set1', ax=ax3)
        ax3.set_title('Unique Sound Classes by House', fontsize=12, fontweight='bold')
        ax3.set_xlabel('House ID', fontsize=10)
        ax3.set_ylabel('Number of Unique Classes', fontsize=10)
        ax3.tick_params(axis='x', rotation=45)
        
        # Scatter plot: Detections vs Confidence
        sns.scatterplot(data=house_df, x='TOTAL_DETECTIONS', y='AVG_CONFIDENCE', 
                       size='UNIQUE_CLASSES', sizes=(100, 500), ax=ax4)
        ax4.set_title('House Characteristics: Detections vs Confidence', fontsize=12, fontweight='bold')
        ax4.set_xlabel('Total Detections', fontsize=10)
        ax4.set_ylabel('Average Confidence', fontsize=10)
        
        plt.suptitle('Spatial Distribution Analysis - House Comparison', 
                    fontsize=16, fontweight='bold', y=0.98)
        plt.tight_layout()
        plt.savefig('spatial_analysis.png', dpi=300, bbox_inches='tight')
        plt.show()
    
    # 5. ROOM COMPARISON
    print("5. Creating room comparison visualizations...")
    
    # Room-specific sound patterns
    room_class_data = df.group_by("ROOM_NAME", "CLASS_NAME").agg(
        F.count("*").alias("FREQUENCY")
    ).collect()
    
    if room_class_data:
        room_class_df = pd.DataFrame([row.asDict() for row in room_class_data])
        
        # Get top classes for each room
        top_kitchen = room_class_df[room_class_df['ROOM_NAME'] == 'Kitchen'].nlargest(10, 'FREQUENCY')
        top_living = room_class_df[room_class_df['ROOM_NAME'] == 'Living Room'].nlargest(10, 'FREQUENCY')
        
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))
        
        # Kitchen top sounds
        if not top_kitchen.empty:
            sns.barplot(data=top_kitchen, y='CLASS_NAME', x='FREQUENCY', 
                       palette='Oranges_r', ax=ax1)
            ax1.set_title('Top 10 Sound Classes in Kitchen', fontsize=14, fontweight='bold')
            ax1.set_xlabel('Number of Detections', fontsize=12)
            ax1.set_ylabel('Sound Class', fontsize=12)
            ax1.set_xscale('log')
        
        # Living room top sounds
        if not top_living.empty:
            sns.barplot(data=top_living, y='CLASS_NAME', x='FREQUENCY', 
                       palette='Blues_r', ax=ax2)
            ax2.set_title('Top 10 Sound Classes in Living Room', fontsize=14, fontweight='bold')
            ax2.set_xlabel('Number of Detections', fontsize=12)
            ax2.set_ylabel('Sound Class', fontsize=12)
            ax2.set_xscale('log')
        
        plt.suptitle('Room-Specific Sound Event Patterns', fontsize=16, fontweight='bold')
        plt.tight_layout()
        plt.savefig('room_comparison.png', dpi=300, bbox_inches='tight')
        plt.show()
    
    # 6. PARENT CATEGORY ANALYSIS
    print("6. Creating parent category visualizations...")
    
    parent_data = df.filter(F.col("PARENT_NAME").isNotNull()).group_by("PARENT_NAME").agg(
        F.count("*").alias("FREQUENCY"),
        F.avg("PROBABILITY").alias("AVG_CONFIDENCE")
    ).order_by(F.desc("FREQUENCY")).limit(15).collect()
    
    if parent_data:
        parent_df = pd.DataFrame([row.asDict() for row in parent_data])
        parent_df['PERCENTAGE'] = (parent_df['FREQUENCY'] / total_rows) * 100
        
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 8))
        
        # Frequency plot
        sns.barplot(data=parent_df, y='PARENT_NAME', x='FREQUENCY', 
                   palette='viridis', ax=ax1)
        ax1.set_title('Top 15 Parent Categories by Frequency', fontsize=14, fontweight='bold')
        ax1.set_xlabel('Number of Detections (log scale)', fontsize=12)
        ax1.set_ylabel('Parent Category', fontsize=12)
        ax1.set_xscale('log')
        
        # Donut chart for top categories
        colors = sns.color_palette("husl", len(parent_df))
        wedges, texts, autotexts = ax2.pie(parent_df['FREQUENCY'], labels=parent_df['PARENT_NAME'], 
                                          autopct='%1.1f%%', colors=colors, pctdistance=0.85)
        centre_circle = plt.Circle((0,0), 0.70, fc='white')
        ax2.add_artist(centre_circle)
        ax2.set_title('Distribution of Parent Categories', fontsize=14, fontweight='bold')
        
        # Adjust text size
        for autotext in autotexts:
            autotext.set_fontsize(8)
        for text in texts:
            text.set_fontsize(8)
        
        plt.tight_layout()
        plt.savefig('parent_categories.png', dpi=300, bbox_inches='tight')
        plt.show()
    
    print("Generated plots:")
    print("1. temporal_heatmap.png - Daily activity patterns")
    print("2. sound_class_analysis.png - Sound class distribution and confidence")
    print("3. confidence_analysis.png - Comprehensive confidence score analysis")
    print("4. spatial_analysis.png - House-level comparisons")
    print("5. room_comparison.png - Room-specific sound patterns")
    print("6. parent_categories.png - Parent category analysis")


# Additional utility functions for extended analysis
def export_summary_statistics():
    """Export key statistics"""
    df = session.table("vitalise_data_light_01")
    
    # Create comprehensive summary table
    summary_stats = df.select(
        F.count("*").alias("total_records"),
        F.count_distinct("HOUSE_NAME").alias("unique_houses"),
        F.count_distinct("ROOM_NAME").alias("unique_rooms"),
        F.count_distinct("CLASS_NAME").alias("unique_sound_classes"),
        F.min("PROBABILITY").alias("min_confidence"),
        F.max("PROBABILITY").alias("max_confidence"),
        F.avg("PROBABILITY").alias("mean_confidence"),
        F.stddev("PROBABILITY").alias("std_confidence")
    ).collect()
    
    return summary_stats

def analyze_sound_event_sequences():
    """Analyze sequential patterns in sound events - useful for ADL research"""
    df = session.table("vitalise_data_light_01")
    
    # Get sequences of sound events within each file
    sequences = df.select(
        "FILENAME", "HOUSE_NAME", "ROOM_NAME", 
        "FRAME_INDEX", "CLASS_NAME", "PROBABILITY"
    ).order_by("FILENAME", "FRAME_INDEX").collect()
    
    return sequences

# Run the main EDA function
if __name__ == "__main__":
    main()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from snowflake.snowpark import functions as F

# Load the table
df = session.table("vitalise_data_light_01")

# Extract hour from TIME_COL and count events
hourly_counts = (
    df.select(F.hour(F.to_time("TIME_COL")).alias("HOUR"))
      .group_by("HOUR")
      .agg(F.count("*").alias("DETECTIONS"))
      .order_by("HOUR")
      .to_pandas()
)

# Plot
plt.figure(figsize=(10,6))
sns.barplot(x="HOUR", y="DETECTIONS", data=hourly_counts, color="steelblue")
plt.title("Distribution of Sound Event Detections by Hour of Day", fontsize=14)
plt.xlabel("Hour of Day (0–23)")
plt.ylabel("Number of Detections")
plt.xticks(range(0,24))
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from snowflake.snowpark import functions as F

# Load table
df = session.table("vitalise_data_light_01")

# Hourly distribution per house
house_hourly = (
    df.select(
        F.col("HOUSE_NAME"),
        F.hour(F.to_time("TIME_COL")).alias("HOUR")
    )
    .group_by("HOUSE_NAME", "HOUR")
    .agg(F.count("*").alias("DETECTIONS"))
    .order_by("HOUSE_NAME", "HOUR")
    .to_pandas()
)

# Plot: heatmap of hours vs houses
pivot_table = house_hourly.pivot(index="HOUSE_NAME", columns="HOUR", values="DETECTIONS").fillna(0)

plt.figure(figsize=(14,6))
sns.heatmap(
    pivot_table, 
    cmap="YlGnBu", 
    cbar_kws={'label': 'Number of Detections'}, 
    linewidths=0.3
)
plt.title("Hourly Distribution of Recordings by House", fontsize=14)
plt.xlabel("Hour of Day (0–23)")
plt.ylabel("House")
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()
